# 0.&nbsp;Setup Environment

## 0.1 Install Packages

In [ ]:
!pip install --quiet duckdb
!pip install --quiet jupysql
!pip install --quiet duckdb-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.2/303.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.1 MB/s eta 0:00:00


## 0.2 Connect G-Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
default_dir = "/content/drive/MyDrive/.../sql_homework_setup" #Sesuaikan!
os.chdir(default_dir)

In [ ]:
# Pastikan Muncul order_table.csv, user_table.csv,
# dan "[To Be Shared - Solution] SQL Homework.ipynb" (atau nama file colab anda!)
!ls

 order_table.csv		      sql_homework.db	   'SQL Homework.ipynb'
'Petunjuk Pengerjaan Homework.gdoc'   sql_homework.db.wal   user_table.csv


## 0.3 Library Import and Configuration

In [ ]:
import duckdb
import pandas as pd
# No need to import sqlalchemy or duckdb_engine
# JupySQL will use SQLAlchemy to auto-detect the driver needed based on your connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

We configure jupysql to return data as a Pandas dataframe and have less verbose output

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = True

## 0.4 Connecting to DuckDB
Connect jupysql to DuckDB using a SQLAlchemy-style connection string. You may either connect to an in memory DuckDB, or a file backed db.

### Create DB connection

In [ ]:
# Create 'sql_homework.db' assuming 'sql_homework.db' is not exist!
# If already exist in default directory, please delete 'sql_homework.db' first!
%sql duckdb:///sql_homework.db

In [ ]:
# create connection to 'sql_homework.db'
conn = duckdb.connect('sql_homework.db')

### Test DuckDB Engine

In [ ]:
# test DuckDB
%sql SELECT 'Off and flying!' AS a_duckdb_column

,a_duckdb_column
0,Off and flying!


In [ ]:
%%sql
SELECT
  'Off and flying!' AS a_duckdb_column

,a_duckdb_column
0,Off and flying!


### CREATE TABLE on 'sql_homework.db'

In [ ]:
# Create new table 'user_table' from 'user_table.csv'
# will getting error if sql_homework.db already exists in the path directory (just continue/ignore the error!)
%%sql
CREATE TABLE user_table AS (
  SELECT
    *
  FROM
    read_csv_auto('user_table.csv')
);

RuntimeError: (duckdb.CatalogException) Catalog Error: Table with name "user_table" already exists!
[SQL: CREATE TABLE user_table AS (
  SELECT
    *
  FROM
    read_csv_auto('user_table.csv')
);]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [ ]:
# Create new table 'user_table' from 'user_table.csv'
# will getting error if sql_homework.db already exists in the path directory (just continue/ignore the error!)
%%sql
CREATE TABLE order_table AS (
  SELECT
    *
  FROM
    read_csv_auto('order_table.csv')
);

RuntimeError: (duckdb.CatalogException) Catalog Error: Table with name "order_table" already exists!
[SQL: CREATE TABLE order_table AS (
  SELECT
    *
  FROM
    read_csv_auto('order_table.csv')
);]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


### Check if all tables already loaded!

In [ ]:
conn.sql('SHOW ALL TABLES')

┌──────────────┬─────────┬─────────────┬──────────────────────────┬────────────────────────────────────────┬───────────┐
│   database   │ schema  │    name     │       column_names       │              column_types              │ temporary │
│   varchar    │ varchar │   varchar   │        varchar[]         │               varchar[]                │  boolean  │
├──────────────┼─────────┼─────────────┼──────────────────────────┼────────────────────────────────────────┼───────────┤
│ sql_homework │ main    │ order_table │ [order_id, user_id, it…  │ [BIGINT, BIGINT, BIGINT, DOUBLE, DATE] │ false     │
│ sql_homework │ main    │ user_table  │ [user_id, register_tim…  │ [BIGINT, DATE, VARCHAR]                │ false     │
└──────────────┴─────────┴─────────────┴──────────────────────────┴────────────────────────────────────────┴───────────┘

In [ ]:
conn.sql('DESCRIBE user_table')

┌───────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│  column_name  │ column_type │  null   │   key   │ default │ extra │
│    varchar    │   varchar   │ varchar │ varchar │ varchar │ int32 │
├───────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ user_id       │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ register_time │ DATE        │ YES     │ NULL    │ NULL    │  NULL │
│ country       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
└───────────────┴─────────────┴─────────┴─────────┴─────────┴───────┘

In [ ]:
conn.sql('DESCRIBE order_table')

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ order_id    │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ user_id     │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ item_id     │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ gmv         │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ order_time  │ DATE        │ YES     │ NULL    │ NULL    │  NULL │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴───────┘

### Testing run query

In [ ]:
%%sql
SELECT
  *
FROM
  order_table
LIMIT
  10;

,order_id,user_id,item_id,gmv,order_time
0,1030132,64177,3366770,27.0,2017-04-24
1,1030137,10475,6130641,69.0,2017-02-02
2,1030147,28286,6770063,87.0,2017-04-25
3,1030153,28282,4193426,82.0,2017-05-11
4,1030155,64970,8825994,29.0,2017-03-07
5,1030160,37113,5660916,44.0,2017-01-30
6,1030168,69199,8181828,25.0,2017-04-24
7,1030170,28074,4100907,53.0,2017-01-22
8,1030174,51228,5068015,31.0,2017-05-20
9,1030177,66749,7899619,57.0,2017-02-25


In [ ]:
%%sql
SELECT
  *
FROM
  user_table
LIMIT
  10;

,user_id,register_time,country
0,10310,2017-03-02,TW
1,10313,2017-03-20,SG
2,10323,2017-03-26,TW
3,10330,2017-05-02,VN
4,10333,2017-01-10,TH
5,10341,2017-03-09,TH
6,10347,2017-03-02,TH
7,10353,2017-04-27,SG
8,10360,2017-05-31,PH
9,10366,2017-03-05,ID


In [ ]:
%config SqlMagic.displaycon = False

# Homework

You are to write SQL statements to answer the following questions, using the 2 tables user_table and order_table. Sample data is present in the respectively named sheets. (100 marks).

You may use the internet for help.

# 1.&nbsp;Task 1


## Question
Write an SQL statement to count the number of users per country (5 marks)

Clue: Learn COUNT(DISTINCT column_name)

Check if all users in column user_id are unique!

In [ ]:
%%sql
SELECT
  COUNT(*) AS row_count,
  COUNT(DISTINCT user_id) AS unique_user_count,
FROM
  user_table;

,row_count,unique_user_count
0,10855,10855


## SQL Query Answers

In [ ]:
%%sql
SELECT
  country,
  COUNT(user_id) AS user_count
FROM
  user_table
GROUP BY
  country
ORDER BY
  user_count DESC;

,country,user_count
0,VN,1605
1,PH,1583
2,SG,1562
3,TW,1539
4,ID,1531
5,MY,1528
6,TH,1507


# 2.&nbsp;Task 2


## Question
Write an SQL statement to count the number of orders and it’s GMV per country (10 marks)

Check if all order_id in the order_table are unique!

In [ ]:
%%sql
SELECT
  COUNT(*) AS row_count,
  COUNT(DISTINCT item_id) AS unique_item_count,
  COUNT(DISTINCT order_id) AS unique_order_count
FROM
  order_table;

,row_count,unique_item_count,unique_order_count
0,30041,29984,30041


## SQL Query Answers

In [ ]:
%%sql
SELECT
  u.country,
  COUNT(DISTINCT o.order_id) AS country_total_order,
  SUM(o.gmv) AS country_total_gmv
FROM
  order_table o
LEFT JOIN
  user_table u
ON
  o.user_id = u.user_id
GROUP BY
  u.country;

,country,country_total_order,country_total_gmv
0,PH,4350,222450.0
1,ID,4412,222120.0
2,TW,4403,224077.0
3,MY,4173,210078.0
4,TH,4169,210424.0
5,SG,4167,211519.0
6,VN,4367,220512.0


# 3.&nbsp;Task 3


## Question
Write an SQL statement to total unique user doing transaction each month (15 marks)

Check if we have one or more than one year data available!

In [ ]:
%%sql
SELECT
  DISTINCT order_year AS unique_year_available
FROM (
  SELECT
    DATE_PART('year', order_time) AS order_year
  FROM
    order_table
  )
GROUP BY
  order_year;

,unique_year_available
0,2017


## SQL Query Answers

### Simple Way

In [ ]:
%%sql
WITH month_order AS (
  SELECT
    user_id,
    DATE_PART('month', order_time) AS order_month,
  FROM
    order_table
)

SELECT
  order_month,
  COUNT(DISTINCT user_id) AS unique_user_trx
FROM
  month_order
GROUP BY
  order_month
ORDER BY
  order_month ASC;

,order_month,unique_user_trx
0,1,4666
1,2,4343
2,3,4644
3,4,4588
4,5,4657
5,6,205


### Direct Way

In [ ]:
%%sql
SELECT
  DATE_PART('month', order_time) AS order_month,
  COUNT(DISTINCT user_id) AS unique_user_trx
FROM
  order_table
GROUP BY
  order_month
ORDER BY
  order_month ASC;

,order_month,unique_user_trx
0,1,4666
1,2,4343
2,3,4644
3,4,4588
4,5,4657
5,6,205


# 4.&nbsp;Task 4


## Question
Write an SQL statement to find the number of users who made their first order in each country, each month. Also order them by country (ASC), then by month (DESC) (20 marks)

Clue: Create CTE contains user_id and month of their first order! (use MIN aggregation on DATE_PART('month', order_time))

In [ ]:
%%sql
SELECT
  user_id,
  MIN(DATE_PART('month', order_time)) AS month_of_first_order
FROM
  order_table
GROUP BY
  user_id;

,user_id,month_of_first_order
0,64177,4
1,28286,2
2,28282,2
3,33156,1
4,56065,3
...,...,...
10176,44951,4
10177,49213,5
10178,66581,3
10179,20507,1


## SQL Query Answers

In [ ]:
%%sql
WITH user_first_order AS (
  SELECT
    user_id,
    MIN(DATE_PART('month', order_time)) AS month_of_first_order
  FROM
    order_table
  GROUP BY
    user_id
)

SELECT
  b.country,
  a.month_of_first_order,
  COUNT(a.user_id) AS user_count
FROM
  user_first_order a
LEFT JOIN
  user_table b
ON
  a.user_id = b.user_id
GROUP BY
  b.country,
  a.month_of_first_order
ORDER BY
  b.country ASC,
  a.month_of_first_order ASC;

# 5.&nbsp;Task 5


## Question
Write an SQL statement to find the first order GMV of each user. If there is a tie, use the order with the lower order_id (20 marks)

Clue: Create CTE-1 to contains user_id and first_order_date -> Use the CTE-1 to create CTE-2 contains user_id and first_order_id -> Use CTE-2 to answer!

In [ ]:
%%sql
SELECT
  user_id,
  MIN(order_time) AS first_order_date
FROM
  order_table
GROUP BY
  user_id
ORDER BY
  first_order_date ASC,
  user_id ASC;

,user_id,first_order_date
0,10565,2017-01-01
1,11278,2017-01-01
2,11367,2017-01-01
3,11501,2017-01-01
4,11526,2017-01-01
...,...,...
10176,49245,2017-06-01
10177,60541,2017-06-01
10178,61398,2017-06-01
10179,65581,2017-06-01


## SQL Query Answers

### Simple Way

In [ ]:
%%sql
WITH user_first_order_time AS (
  SELECT
    user_id,
    MIN(order_time) AS first_order_date
  FROM
    order_table
  GROUP BY
    user_id
),

user_first_order_id AS (
  SELECT
    a.user_id,
    MIN(b.order_id) AS first_order_id
  FROM
    user_first_order_time a
  LEFT JOIN
    order_table b
  ON
    a.user_id = b.user_id
    AND a.first_order_date = b.order_time
  GROUP BY
    a.user_id,
    a.first_order_date
)

SELECT
  a.user_id,
  b.gmv
FROM
  user_first_order_id a
LEFT JOIN
  order_table b
ON
  a.user_id = b.user_id
ORDER BY
  b.gmv DESC,
  a.user_id ASC;

,user_id,gmv
0,10399,100.0
1,10614,100.0
2,10672,100.0
3,10681,100.0
4,10746,100.0
...,...,...
30036,69287,1.0
30037,69389,1.0
30038,69389,1.0
30039,69619,1.0


### With Sub-Query

In [ ]:
%%sql
SELECT
  * EXCLUDE(row_num)
FROM (
  SELECT
    user_id,
    gmv,
    ROW_NUMBER() OVER (PARTITION BY order_time, order_id) AS row_num
  FROM
    order_table
)
WHERE
  row_num = 1
ORDER BY
  gmv DESC,
  user_id ASC;

,user_id,gmv
0,10399,100.0
1,10614,100.0
2,10672,100.0
3,10681,100.0
4,10746,100.0
...,...,...
30036,69287,1.0
30037,69389,1.0
30038,69389,1.0
30039,69619,1.0


### Advanced Level Query

In [ ]:
%%sql
SELECT
  user_id,
  gmv
FROM
  order_table
QUALIFY
  ROW_NUMBER() OVER (PARTITION BY order_time, order_id) = 1
ORDER BY
  gmv DESC,
  user_id ASC;

,user_id,gmv
0,10399,100.0
1,10614,100.0
2,10672,100.0
3,10681,100.0
4,10746,100.0
...,...,...
30036,69287,1.0
30037,69389,1.0
30038,69389,1.0
30039,69619,1.0


#6.&nbsp;Task 6

## Questions
A star user is the one who have GMV more than $100.0 a month. Write an SQL statement to find all star user ordered by the total month he/she become a star user (30 marks).

## SQL Query Answers

In [ ]:
%%sql
WITH month_of_order AS (
SELECT
  user_id,
  DATE_PART('month', order_time) AS order_month,
  gmv
FROM
  order_table
),

user_months_of_star AS (
  SELECT
    user_id,
    order_month,
    SUM(gmv) AS monthly_gmv
  FROM
    month_of_order
  GROUP BY
    user_id,
    order_month
  HAVING
    monthly_gmv >= 100
)

SELECT
  user_id,
  COUNT(order_month) AS stars_count
FROM
  user_months_of_star
GROUP BY
  user_id
ORDER BY
  stars_count DESC,
  user_id ASC;

,user_id,stars_count
0,68262,4
1,10341,3
2,17508,3
3,18331,3
4,19085,3
...,...,...
3051,70200,1
3052,70219,1
3053,70239,1
3054,70248,1


# Close Connection

In [ ]:
conn.close()